In [1]:
import os
os.chdir("/home/composersyf/Documents/RelishMBA/MixPanel_Data")

In [2]:
import json

with open("All_Events_Mixpanel_2017_02_28.json") as events_data:
    events_data=events_data.readlines()[0]
events=json.loads(events_data) #events is stored as a list

with open("All_People_Mixpanel_2017_02_28.json") as people_data:
    people_data=people_data.readlines()[0]
people=json.loads(people_data) #people is also stored as a list

In [3]:
import numpy as np
import pandas as pd
import re
from collections import Counter

In [4]:
# Fix the data problems (company having multiple names) within the Events Data
for k,e in enumerate(events):
    try:
        if e['properties']['company_name']=="PwC's Strategy&" or e['properties']['company_name']=="PwC Consulting":
            events[k]['properties']['company_name']="PwC"
        elif e['properties']['company_name']=="Nike":
            events[k]['properties']['company_name']="Nike Inc."
        elif e['properties']['company_name'] in ['JP Morgan Chase & Co.', 'JP Morgan']:
            events[k]['properties']['company_name']='JPMorgan Chase & Co. Global Wealth Management & Chase Leaders'
        elif e['properties']['company_name']=="Nestle":
            events[k]['properties']['company_name']="Nestlé USA"
        elif e['properties']['company_name']=='Siemens Healthcare':
            events[k]['properties']['company_name']='Siemens Healthineers'
        elif e['properties']['company_name']=='E&J Gallo':
            events[k]['properties']['company_name']='E. & J. Gallo Winery'
    except KeyError:
        pass
    try:
        if e['properties']['name']=="Premier":
            events[k]['properties']['name']="Premier, Inc."
        elif e['properties']['name']=="PwC's Strategy&" or e['properties']['name']=="PwC Consulting":
            events[k]['properties']['name']="PwC"
        elif e['properties']['name']=="Nike":
            events[k]['properties']['name']="Nike Inc."
        elif e['properties']['name'] in ['JP Morgan Chase & Co.', 'JP Morgan']:
            events[k]['properties']['name']='JPMorgan Chase & Co. Global Wealth Management & Chase Leaders'
        elif e['properties']['name']=="Nestle":
            events[k]['properties']['name']="Nestlé USA"
        elif e['properties']['name']=='Siemens Healthcare':
            events[k]['properties']['name']='Siemens Healthineers'
        elif e['properties']['name']=='E&J Gallo':
            events[k]['properties']['name']='E. & J. Gallo Winery'
        elif e['properties']['name']=='WillowTree':
            events[k]['properties']['name']='WillowTree, Inc.'
    except KeyError:
        pass

In [5]:
len(events) #total number of events so far (as of 01/31/2017)

207281

# Company Page (Site) Popularity

### (0) Build a dictionary to map company ids to company names, and the inversed dictionary

In [6]:
company_ids_names=[]
for e in events:
    if e['name']=="view Profile":
        company_ids_names.append((e['properties']["company_id"],e['properties']['company_name']))
company_ids_names_dict={}
for pair in company_ids_names:
    try:
        if pair[1] not in company_ids_names_dict[pair[0]]:
            company_ids_names_dict[pair[0]].append(pair[1])
    except KeyError:
        company_ids_names_dict[pair[0]]=[pair[1]]
company_ids_names_dict

{13: ['Bain & Company'],
 14: ['3M'],
 15: ['DuPont'],
 16: ['Corning Inc.'],
 17: ['Accenture'],
 18: ['RelishMBA'],
 20: ['Harris Williams'],
 23: ['PwC'],
 24: ['Consulting Industry'],
 25: ['Finance Industry'],
 27: ['Technology Industry'],
 28: ['Consumer Products Industry'],
 30: ['Walmart'],
 33: ['Boston Consulting Group'],
 37: ['Mastercard'],
 40: ['Morgan Stanley'],
 42: ['E. & J. Gallo Winery'],
 43: ['McKinsey & Company'],
 46: ['Evercore'],
 50: ['Chevron'],
 54: ['Genentech'],
 56: ['LORD Corporation'],
 57: ['M&T Bank'],
 60: ['Humana'],
 63: ['Emerson'],
 68: ['Crane'],
 71: ['BB&T'],
 72: ['Ford Motor Company'],
 74: ['Cigna'],
 75: ['Under Armour'],
 78: ['Peer Insight'],
 81: ['BizCorps'],
 84: ['Forcier Consulting'],
 88: ['KiwiTech'],
 90: ['Alcoa Inc.'],
 96: ['Greystar'],
 98: ['Livingstone Partners'],
 99: ['National Microfinance Bank'],
 100: ['Nike Inc.'],
 103: ['Qualtrics'],
 105: ['Samsung Electronics America'],
 109: ['Zillow'],
 110: ['Jet.com'],
 112: [

In [7]:
#check if there are companies who changed their names displayed in RelishMBA platform ('properties'--'company name')
for k in company_ids_names_dict:
    if len(company_ids_names_dict[k])>1:
        print(k,company_ids_names_dict[k])

In [8]:
company_ids_names_2=[]
for e in events:
    if e['name']=="Companny page quick link visit":
        company_ids_names_2.append((e['properties']["company_id"],e['properties']['name']))
company_ids_names_dict_2={}
for pair in company_ids_names_2:
    try:
        if pair[1] not in company_ids_names_dict_2[pair[0]]:
            company_ids_names_dict_2[pair[0]].append(pair[1])
    except KeyError:
        company_ids_names_dict_2[pair[0]]=[pair[1]]
company_ids_names_dict_2

{'100': ['Nike Inc.'],
 '103': ['Qualtrics'],
 '105': ['Samsung Electronics America'],
 '109': ['Zillow'],
 '110': ['Jet.com'],
 '112': ['Siemens Healthineers'],
 '113': ['The Beacon Group'],
 '117': ['Premier, Inc.'],
 '118': ['Privia Health'],
 '119': ['Reckitt Benckiser (RB)'],
 '120': ['Red Ventures'],
 '122': ['Thornburg Investment Management'],
 '126': ['American Express'],
 '13': ['Bain & Company'],
 '134': ["L'Oréal"],
 '138': ['Red Hat'],
 '14': ['3M'],
 '145': ['Environmental Defense Fund (EDF)'],
 '148': ['Nestlé USA'],
 '149': ['Palantir Technologies'],
 '15': ['DuPont'],
 '159': ['VinConnect Inc.'],
 '16': ['Corning Inc.'],
 '160': ['Archer Roose'],
 '162': ['WorkGenius'],
 '163': ['Castle Hill Gaming'],
 '164': ['ArcheMedX'],
 '165': ['Guesthouse Reservation Services (Cassimir Club)'],
 '171': ['LLJ Capital'],
 '178': ['RBC'],
 '179': ['Split'],
 '18': ['RelishMBA'],
 '182': ['JPMorgan Chase & Co. Global Wealth Management & Chase Leaders'],
 '188': ['Medtronic'],
 '189': 

In [9]:
#check if there are companies who changed their names displayed in RelishMBA ('properties'--'name')
for k in company_ids_names_dict_2:
    if len(company_ids_names_dict_2[k])>1:
        print (k,company_ids_names_dict_2[k])

In [10]:
#check if the company names in dict_2 is consistent with the company names in dict_1:
dict_names=list(company_ids_names_dict.values())
for k in company_ids_names_dict_2:
    if company_ids_names_dict_2[k] not in dict_names:
        print(company_ids_names_dict_2[k])
        try:
            print(company_ids_names_dict[int(k)])
        except KeyError:
            pass

['A.T. Kearney']


In [11]:
#inverse dictionary
company_names_ids_dict={}
for pair in company_ids_names:
    company_names_ids_dict[pair[1]]=pair[0]
company_names_ids_dict

{'3M': 14,
 'Accenture': 17,
 'Affinity Management': 214,
 'Alcoa Inc.': 90,
 'American Express': 126,
 'ArcheMedX': 164,
 'Archer Roose': 160,
 'BB&T': 71,
 'Bain & Company': 13,
 'Barclays': 193,
 'BizCorps': 81,
 'Boston Consulting Group': 33,
 'Castle Hill Gaming': 163,
 'Chevron': 50,
 'Cigna': 74,
 'Citi': 191,
 'Consulting Industry': 24,
 'Consumer Products Industry': 28,
 'Corning Inc.': 16,
 'Crane': 68,
 'DaVita Healthcare Partners': 190,
 'DuPont': 15,
 'E. & J. Gallo Winery': 42,
 'Emerson': 63,
 'Environmental Defense Fund (EDF)': 145,
 'Evercore': 46,
 'Falconi': 207,
 'Finance Industry': 25,
 'Forcier Consulting': 84,
 'Ford Motor Company': 72,
 'Genentech': 54,
 'Greystar': 96,
 'Guesthouse Reservation Services (Cassimir Club)': 165,
 'Guggenheim Partners': 212,
 'Harris Williams': 20,
 'Humana': 60,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 182,
 'Jet.com': 110,
 'KiwiTech': 88,
 "L'Oréal": 134,
 'LLJ Capital': 171,
 'LORD Corporation': 56,
 'Li

In [12]:
len(company_names_ids_dict) #total number of companies in the system

78

In [13]:
#The companies that need to be excluded
exclude_list=['3M','Accenture','Barclays','Boston Consulting Group','Citi','DaVita Healthcare Partners','Guggenheim Partners','Microsoft']
for e in exclude_list:
    del company_names_ids_dict[e]
company_names_ids_dict

{'Affinity Management': 214,
 'Alcoa Inc.': 90,
 'American Express': 126,
 'ArcheMedX': 164,
 'Archer Roose': 160,
 'BB&T': 71,
 'Bain & Company': 13,
 'BizCorps': 81,
 'Castle Hill Gaming': 163,
 'Chevron': 50,
 'Cigna': 74,
 'Consulting Industry': 24,
 'Consumer Products Industry': 28,
 'Corning Inc.': 16,
 'Crane': 68,
 'DuPont': 15,
 'E. & J. Gallo Winery': 42,
 'Emerson': 63,
 'Environmental Defense Fund (EDF)': 145,
 'Evercore': 46,
 'Falconi': 207,
 'Finance Industry': 25,
 'Forcier Consulting': 84,
 'Ford Motor Company': 72,
 'Genentech': 54,
 'Greystar': 96,
 'Guesthouse Reservation Services (Cassimir Club)': 165,
 'Harris Williams': 20,
 'Humana': 60,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 182,
 'Jet.com': 110,
 'KiwiTech': 88,
 "L'Oréal": 134,
 'LLJ Capital': 171,
 'LORD Corporation': 56,
 'Livingstone Partners': 98,
 'M&T Bank': 57,
 'Mastercard': 37,
 'McKinsey & Company': 43,
 'Medtronic': 188,
 'Morgan Stanley': 40,
 'Morningstar': 116,
 'Natio

In [14]:
len(company_names_ids_dict) #number of active companies on the RelishMBA site

70

In [15]:
#save the latest company names as well as their corresponding ids to file
pd.DataFrame(list(company_names_ids_dict.items())).sort_values([0]).rename({0:"company_name",1:"company_id"}).to_csv("company_id.csv",index=False)

### (1) Company Page Number of Visits by Student Users

In [16]:
#main page total number of visits
company_page_visits={}
for e in events:
    if e["name"]=="view Profile":
        try:
            company_page_visits[e["properties"]["company_name"]]+=1
        except KeyError:
            company_page_visits[e["properties"]["company_name"]]=1
company_page_visits

{'3M': 430,
 'Accenture': 1,
 'Affinity Management': 164,
 'Alcoa Inc.': 93,
 'American Express': 762,
 'ArcheMedX': 37,
 'Archer Roose': 49,
 'BB&T': 259,
 'Bain & Company': 913,
 'Barclays': 1,
 'BizCorps': 264,
 'Boston Consulting Group': 2,
 'Castle Hill Gaming': 38,
 'Chevron': 386,
 'Cigna': 325,
 'Citi': 1,
 'Consulting Industry': 404,
 'Consumer Products Industry': 330,
 'Corning Inc.': 221,
 'Crane': 113,
 'DaVita Healthcare Partners': 1,
 'DuPont': 277,
 'E. & J. Gallo Winery': 292,
 'Emerson': 227,
 'Environmental Defense Fund (EDF)': 186,
 'Evercore': 261,
 'Falconi': 129,
 'Finance Industry': 154,
 'Forcier Consulting': 67,
 'Ford Motor Company': 161,
 'Genentech': 578,
 'Greystar': 37,
 'Guesthouse Reservation Services (Cassimir Club)': 28,
 'Guggenheim Partners': 1,
 'Harris Williams': 177,
 'Humana': 179,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 351,
 'Jet.com': 38,
 'KiwiTech': 136,
 "L'Oréal": 324,
 'LLJ Capital': 62,
 'LORD Corporation': 74,


In [17]:
# main page total number of visits by distinct user id
company_page_people_visits={}
for e in events:
    if e["name"]=="view Profile":
        try:
            company_page_people_visits[e["properties"]["company_name"]].append(e["distinct_id"])
        except KeyError:
            company_page_people_visits[e["properties"]["company_name"]]=[e["distinct_id"]]
for k in company_page_people_visits:
    company_page_people_visits[k]=len(set(company_page_people_visits[k]))
company_page_people_visits

{'3M': 274,
 'Accenture': 1,
 'Affinity Management': 132,
 'Alcoa Inc.': 23,
 'American Express': 397,
 'ArcheMedX': 24,
 'Archer Roose': 34,
 'BB&T': 199,
 'Bain & Company': 666,
 'Barclays': 1,
 'BizCorps': 219,
 'Boston Consulting Group': 1,
 'Castle Hill Gaming': 28,
 'Chevron': 258,
 'Cigna': 216,
 'Citi': 1,
 'Consulting Industry': 343,
 'Consumer Products Industry': 207,
 'Corning Inc.': 180,
 'Crane': 84,
 'DaVita Healthcare Partners': 1,
 'DuPont': 166,
 'E. & J. Gallo Winery': 205,
 'Emerson': 134,
 'Environmental Defense Fund (EDF)': 144,
 'Evercore': 206,
 'Falconi': 100,
 'Finance Industry': 125,
 'Forcier Consulting': 60,
 'Ford Motor Company': 118,
 'Genentech': 280,
 'Greystar': 23,
 'Guesthouse Reservation Services (Cassimir Club)': 18,
 'Guggenheim Partners': 1,
 'Harris Williams': 126,
 'Humana': 124,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 248,
 'Jet.com': 27,
 'KiwiTech': 116,
 "L'Oréal": 194,
 'LLJ Capital': 42,
 'LORD Corporation': 55,
 

In [18]:
#quick link total number of visits
quick_link_visits={}
for e in events:
    if e["name"]=="Companny page quick link visit":
        try:
            quick_link_visits[e["properties"]["name"]]+=1
        except KeyError:
            quick_link_visits[e["properties"]["name"]]=1
quick_link_visits

{'3M': 94,
 'A.T. Kearney': 2,
 'Affinity Management': 13,
 'Alcoa Inc.': 11,
 'American Express': 72,
 'ArcheMedX': 3,
 'Archer Roose': 4,
 'BB&T': 49,
 'Bain & Company': 103,
 'BizCorps': 6,
 'Castle Hill Gaming': 5,
 'Chevron': 18,
 'Cigna': 52,
 'Consulting Industry': 62,
 'Consumer Products Industry': 68,
 'Corning Inc.': 45,
 'Crane': 29,
 'DuPont': 37,
 'E. & J. Gallo Winery': 79,
 'Emerson': 34,
 'Environmental Defense Fund (EDF)': 36,
 'Evercore': 34,
 'Falconi': 12,
 'Finance Industry': 14,
 'Forcier Consulting': 5,
 'Ford Motor Company': 35,
 'Genentech': 34,
 'Greystar': 7,
 'Guesthouse Reservation Services (Cassimir Club)': 1,
 'Harris Williams': 17,
 'Humana': 5,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 45,
 'Jet.com': 6,
 'KiwiTech': 13,
 "L'Oréal": 36,
 'LLJ Capital': 8,
 'LORD Corporation': 18,
 'Livingstone Partners': 2,
 'M&T Bank': 8,
 'Mastercard': 15,
 'McKinsey & Company': 14,
 'Medtronic': 63,
 'Morgan Stanley': 63,
 'National Microfinan

### (2) Save & Unsave Company Profiles

In [19]:
#find out all student user ids that have saved the company, for each company
company_saved_student_ids={}
for e in events:
    if e["name"]=="Company Save":
        try:
            company_saved_student_ids[e["properties"]["company_name"]].append([e["distinct_id"],e["time"]])
        except KeyError:
            company_saved_student_ids[e["properties"]["company_name"]]=[[e["distinct_id"],e["time"]]]

In [20]:
#count the number of distinct student users who have ever saved the company
company_saved_times={}
for k in company_saved_student_ids:
    df=pd.DataFrame(company_saved_student_ids[k]).sort_values([0,1],ascending=[True,False]).rename(columns={0:"user_id",1:"time"}).groupby(['user_id'],as_index=False).first()
    company_saved_student_ids[k]=df
    company_saved_times[k]=df.shape[0]
company_saved_times

{'3M': 115,
 'Affinity Management': 11,
 'Alcoa Inc.': 4,
 'American Express': 120,
 'ArcheMedX': 4,
 'Archer Roose': 6,
 'BB&T': 57,
 'Bain & Company': 310,
 'BizCorps': 46,
 'Castle Hill Gaming': 2,
 'Chevron': 113,
 'Cigna': 72,
 'Consulting Industry': 130,
 'Consumer Products Industry': 75,
 'Corning Inc.': 69,
 'Crane': 36,
 'DuPont': 74,
 'E. & J. Gallo Winery': 64,
 'Emerson': 61,
 'Environmental Defense Fund (EDF)': 58,
 'Evercore': 86,
 'Falconi': 13,
 'Finance Industry': 51,
 'Forcier Consulting': 18,
 'Ford Motor Company': 69,
 'Genentech': 100,
 'Greystar': 7,
 'Guesthouse Reservation Services (Cassimir Club)': 2,
 'Harris Williams': 41,
 'Humana': 40,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 87,
 'Jet.com': 6,
 'KiwiTech': 22,
 "L'Oréal": 80,
 'LLJ Capital': 9,
 'LORD Corporation': 11,
 'Livingstone Partners': 7,
 'M&T Bank': 30,
 'Mastercard': 28,
 'McKinsey & Company': 23,
 'Medtronic': 64,
 'Morgan Stanley': 119,
 'Morningstar': 2,
 'National Mi

In [21]:
#update the student user_id list by accounting for the students who unsave a company
for e in events:
    if e['name']=="UnSave Company":
        df=company_saved_student_ids[e['properties']['company_name']]
        if df[df.user_id==e["distinct_id"]].shape[0]>0 and df[df.user_id==e["distinct_id"]].iloc[0,1]<e["time"]:
            df=df[df.user_id!=e["distinct_id"]]
        company_saved_student_ids[e['properties']['company_name']]=df

In [22]:
#verify if the users who unsave a company have been accounted for
company_saved_times_updated={}
for k in company_saved_student_ids:
    company_saved_times_updated[k]=company_saved_student_ids[k].shape[0]
company_saved_times_updated

{'3M': 112,
 'Affinity Management': 10,
 'Alcoa Inc.': 4,
 'American Express': 119,
 'ArcheMedX': 4,
 'Archer Roose': 6,
 'BB&T': 55,
 'Bain & Company': 309,
 'BizCorps': 45,
 'Castle Hill Gaming': 2,
 'Chevron': 113,
 'Cigna': 72,
 'Consulting Industry': 129,
 'Consumer Products Industry': 72,
 'Corning Inc.': 68,
 'Crane': 34,
 'DuPont': 74,
 'E. & J. Gallo Winery': 63,
 'Emerson': 61,
 'Environmental Defense Fund (EDF)': 58,
 'Evercore': 86,
 'Falconi': 12,
 'Finance Industry': 51,
 'Forcier Consulting': 18,
 'Ford Motor Company': 67,
 'Genentech': 100,
 'Greystar': 7,
 'Guesthouse Reservation Services (Cassimir Club)': 2,
 'Harris Williams': 41,
 'Humana': 40,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 87,
 'Jet.com': 6,
 'KiwiTech': 22,
 "L'Oréal": 79,
 'LLJ Capital': 9,
 'LORD Corporation': 11,
 'Livingstone Partners': 7,
 'M&T Bank': 30,
 'Mastercard': 28,
 'McKinsey & Company': 23,
 'Medtronic': 64,
 'Morgan Stanley': 119,
 'Morningstar': 2,
 'National Mi

### (3) Follow & Unfollow Company

In [23]:
#find out all student user ids that have followed the company, for each company
company_followed_student_ids={}
for e in events:
    if e["name"]=="Company Follow":
        try:
            company_followed_student_ids[e["properties"]["company_name"]].append([e["distinct_id"],e["time"]])
        except KeyError:
            company_followed_student_ids[e["properties"]["company_name"]]=[[e["distinct_id"],e["time"]]]

In [24]:
#count the number of distinct student users who followed the company
company_followed_times={}
for k in company_followed_student_ids:
    df=pd.DataFrame(company_followed_student_ids[k]).sort_values([0,1],ascending=[True,False]).rename(columns={0:"user_id",1:"time"}).groupby(['user_id'],as_index=False).first()
    company_followed_student_ids[k]=df
    company_followed_times[k]=df.shape[0]
company_followed_times

{'3M': 143,
 'Accenture': 1,
 'Affinity Management': 18,
 'Alcoa Inc.': 6,
 'American Express': 186,
 'ArcheMedX': 5,
 'Archer Roose': 10,
 'BB&T': 74,
 'Bain & Company': 462,
 'BizCorps': 68,
 'Castle Hill Gaming': 3,
 'Chevron': 152,
 'Cigna': 111,
 'Consulting Industry': 246,
 'Consumer Products Industry': 159,
 'Corning Inc.': 91,
 'Crane': 44,
 'DuPont': 85,
 'E. & J. Gallo Winery': 89,
 'Emerson': 73,
 'Environmental Defense Fund (EDF)': 74,
 'Evercore': 93,
 'Falconi': 19,
 'Finance Industry': 119,
 'Forcier Consulting': 48,
 'Ford Motor Company': 102,
 'Genentech': 129,
 'Greystar': 5,
 'Guesthouse Reservation Services (Cassimir Club)': 4,
 'Harris Williams': 47,
 'Humana': 55,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 143,
 'Jet.com': 14,
 'KiwiTech': 41,
 "L'Oréal": 116,
 'LLJ Capital': 13,
 'LORD Corporation': 15,
 'Livingstone Partners': 8,
 'M&T Bank': 47,
 'Mastercard': 34,
 'McKinsey & Company': 30,
 'Medtronic': 112,
 'Morgan Stanley': 167,
 'Mor

In [25]:
#update the student user_id list by accounting for the students who unfollowed a company
for e in events:
    if e['name']=="Company UnFollow":
        df=company_followed_student_ids[e['properties']['company_name']]
        if df[df.user_id==e["distinct_id"]].shape[0]>0 and df[df.user_id==e["distinct_id"]].iloc[0,1]<e["time"]:
            df=df[df.user_id!=e["distinct_id"]]
        company_followed_student_ids[e['properties']['company_name']]=df

In [26]:
#verify if the users who unfollowed a company have been accounted for
company_followed_times_updated={}
for k in company_followed_student_ids:
    company_followed_times_updated[k]=company_followed_student_ids[k].shape[0]
company_followed_times_updated

{'3M': 140,
 'Accenture': 1,
 'Affinity Management': 15,
 'Alcoa Inc.': 6,
 'American Express': 185,
 'ArcheMedX': 5,
 'Archer Roose': 10,
 'BB&T': 71,
 'Bain & Company': 455,
 'BizCorps': 67,
 'Castle Hill Gaming': 3,
 'Chevron': 147,
 'Cigna': 106,
 'Consulting Industry': 242,
 'Consumer Products Industry': 156,
 'Corning Inc.': 88,
 'Crane': 41,
 'DuPont': 84,
 'E. & J. Gallo Winery': 85,
 'Emerson': 73,
 'Environmental Defense Fund (EDF)': 73,
 'Evercore': 91,
 'Falconi': 17,
 'Finance Industry': 117,
 'Forcier Consulting': 45,
 'Ford Motor Company': 100,
 'Genentech': 126,
 'Greystar': 5,
 'Guesthouse Reservation Services (Cassimir Club)': 3,
 'Harris Williams': 47,
 'Humana': 55,
 'JPMorgan Chase & Co. Global Wealth Management & Chase Leaders': 142,
 'Jet.com': 13,
 'KiwiTech': 38,
 "L'Oréal": 115,
 'LLJ Capital': 13,
 'LORD Corporation': 15,
 'Livingstone Partners': 8,
 'M&T Bank': 47,
 'Mastercard': 34,
 'McKinsey & Company': 30,
 'Medtronic': 107,
 'Morgan Stanley': 165,
 'Mor

## Save & Export the Number of Page Visits, Follows and Saves

In [27]:
#As of 02/28/2017, 8 companies out of the 78 signed-up companies are not available to student users
for name in exclude_list:
    try:
        del company_page_visits[name]
    except KeyError:
        pass
for name in exclude_list:
    try:
        del company_page_people_visits[name]
    except KeyError:
        pass
for name in exclude_list:
    try:
        del quick_link_visits[name]
    except KeyError:
        pass
del quick_link_visits['A.T. Kearney']
for name in exclude_list:
    try:
        del company_followed_times_updated[name]
    except KeyError:
        pass
for name in exclude_list:
    try:
        del company_saved_times_updated[name]
    except KeyError:
        pass
print(len(company_page_visits), len(company_page_people_visits), len(quick_link_visits), len(company_followed_times_updated),len(company_saved_times_updated))

70 70 68 70 70


#### Transform the site polularity data which are originally in the dictionary format into pandas DataFrames

In [28]:
page_visits=[]
for k in company_page_visits.keys():
    page_visits.append(k)
    page_visits.append(company_page_visits[k])
page_visits=np.array(page_visits)
page_visits=page_visits.reshape((len(company_page_visits),2))
page_visits=pd.DataFrame(page_visits)
page_visits.columns=["company_name","number_of_visits"]
page_visits=page_visits.sort_values(["company_name"])
page_visits.head(10)

,company_name,number_of_visits
13,Affinity Management,164
9,Alcoa Inc.,93
17,American Express,762
49,ArcheMedX,37
35,Archer Roose,49
51,BB&T,259
52,Bain & Company,913
54,BizCorps,264
0,Castle Hill Gaming,38
66,Chevron,386


In [29]:
people_visits=[]
for k in company_page_people_visits.keys():
    people_visits.append(k)
    people_visits.append(company_page_people_visits[k])
people_visits=np.array(people_visits)
people_visits=people_visits.reshape((len(company_page_people_visits),2))
people_visits=pd.DataFrame(people_visits)
people_visits.columns=["company_name","number_of_distinct_user_visits"]
people_visits=people_visits.sort_values(["company_name"])
people_visits.head(10)

,company_name,number_of_distinct_user_visits
13,Affinity Management,132
9,Alcoa Inc.,23
17,American Express,397
49,ArcheMedX,24
35,Archer Roose,34
51,BB&T,199
52,Bain & Company,666
54,BizCorps,219
0,Castle Hill Gaming,28
66,Chevron,258


In [30]:
link_visits=[]
for k in quick_link_visits.keys():
    link_visits.append(k)
    link_visits.append(quick_link_visits[k])
link_visits=np.array(link_visits)
link_visits=link_visits.reshape((len(quick_link_visits),2))
link_visits=pd.DataFrame(link_visits)
link_visits.columns=["company_name","number_of_quick_link_visits"]
link_visits=link_visits.sort_values(["company_name"])
link_visits.head(10)

,company_name,number_of_quick_link_visits
13,Affinity Management,13
10,Alcoa Inc.,11
45,American Express,72
50,ArcheMedX,3
34,Archer Roose,4
51,BB&T,49
52,Bain & Company,103
37,BizCorps,6
2,Castle Hill Gaming,5
65,Chevron,18


In [31]:
followed=[]
for k in company_followed_times_updated.keys():
    followed.append(k)
    followed.append(company_followed_times_updated[k])
followed=np.array(followed)
followed=followed.reshape((len(company_followed_times_updated),2))
followed=pd.DataFrame(followed)
followed.columns=["company_name","number_of_follows"]
followed=followed.sort_values(["company_name"])
followed.head(10)

,company_name,number_of_follows
11,Affinity Management,15
9,Alcoa Inc.,6
44,American Express,185
48,ArcheMedX,5
33,Archer Roose,10
49,BB&T,71
28,Bain & Company,455
15,BizCorps,67
2,Castle Hill Gaming,3
65,Chevron,147


In [32]:
saved=[]
for k in company_saved_times_updated.keys():
    saved.append(k)
    saved.append(company_saved_times_updated[k])
saved=np.array(saved)
saved=saved.reshape((len(company_saved_times_updated),2))
saved=pd.DataFrame(saved)
saved.columns=["company_name","number_of_saves"]
saved=saved.sort_values(["company_name"])
saved.head(10)

,company_name,number_of_saves
12,Affinity Management,10
10,Alcoa Inc.,4
44,American Express,119
49,ArcheMedX,4
34,Archer Roose,6
51,BB&T,55
52,Bain & Company,309
54,BizCorps,45
2,Castle Hill Gaming,2
66,Chevron,113


In [33]:
final_df=page_visits.merge(people_visits,on=["company_name"],how="outer").merge(link_visits,on=["company_name"],how="outer")\
.merge(followed,on=["company_name"],how="outer").merge(saved,on=["company_name"],how="outer")
final_df.head(10)

,company_name,number_of_visits,number_of_distinct_user_visits,number_of_quick_link_visits,number_of_follows,number_of_saves
0,Affinity Management,164,132,13,15,10
1,Alcoa Inc.,93,23,11,6,4
2,American Express,762,397,72,185,119
3,ArcheMedX,37,24,3,5,4
4,Archer Roose,49,34,4,10,6
5,BB&T,259,199,49,71,55
6,Bain & Company,913,666,103,455,309
7,BizCorps,264,219,6,67,45
8,Castle Hill Gaming,38,28,5,3,2
9,Chevron,386,258,18,147,113


In [34]:
#fill in missing values (this code is subject to change in the future)
#for i in range(final_df.shape[0]):
#    if pd.isnull(final_df.iloc[i,3]):
#        final_df.iloc[i,3]=0
final_df=final_df.fillna(0)

In [35]:
def to_lower(string):
    return string.lower()

In [36]:
final_df=final_df.iloc[np.argsort(np.array(final_df.company_name.apply(to_lower))),:]

In [37]:
#save the cumulative company sites popularity data to file
final_df.to_csv("Company_Sites_Popularity_02_28_2017.csv",index=False)

In [39]:
#import the cumulative company sites popularity data from last month
final_df_0=pd.read_csv("Company_Sites_Popularity_01_31_2017.csv").iloc[:,:]

#final_df_0.ix[np.where(final_df_0.company_name=="Nestle")[0],"company_name"]="Nestlé USA"
#final_df_0.ix[np.where(final_df_0.company_name=="E&J Gallo")[0],"company_name"]="E. & J. Gallo Winery"
#final_df_0.ix[np.where(final_df_0.company_name=="Siemens Healthcare")[0],"company_name"]="Siemens Healthineers"
#final_df_0.ix[np.where(final_df_0.company_name=="JP Morgan")[0],"company_name"]="JPMorgan Chase & Co. Global Wealth Management & Chase Leaders"

final_df_0

,company_name,number_of_visits,number_of_distinct_user_visits,number_of_quick_link_visits,number_of_follows,number_of_saves
0,Affinity Management,110,93,10,11,4
1,Alcoa Inc.,88,22,9,6,4
2,American Express,681,351,58,170,112
3,ArcheMedX,35,22,3,5,3
4,Archer Roose,44,30,4,10,6
5,Bain & Company,836,615,98,433,297
6,BB&T,235,182,44,68,53
7,BizCorps,248,205,6,63,41
8,Castle Hill Gaming,36,27,5,3,2
9,Chevron,349,237,17,141,108


In [40]:
merged_df=final_df_0.merge(final_df,how="outer",on=['company_name'])
merged_df=merged_df.fillna(0)
merged_df

,company_name,number_of_visits_x,number_of_distinct_user_visits_x,number_of_quick_link_visits_x,number_of_follows_x,number_of_saves_x,number_of_visits_y,number_of_distinct_user_visits_y,number_of_quick_link_visits_y,number_of_follows_y,number_of_saves_y
0,Affinity Management,110,93,10,11,4,164,132,13,15,10
1,Alcoa Inc.,88,22,9,6,4,93,23,11,6,4
2,American Express,681,351,58,170,112,762,397,72,185,119
3,ArcheMedX,35,22,3,5,3,37,24,3,5,4
4,Archer Roose,44,30,4,10,6,49,34,4,10,6
5,Bain & Company,836,615,98,433,297,913,666,103,455,309
6,BB&T,235,182,44,68,53,259,199,49,71,55
7,BizCorps,248,205,6,63,41,264,219,6,67,45
8,Castle Hill Gaming,36,27,5,3,2,38,28,5,3,2
9,Chevron,349,237,17,141,108,386,258,18,147,113


In [41]:
final_df_1=pd.DataFrame()
final_df_1['company_name']=np.array(merged_df.company_name)
for i in range(5):
    final_df_1[final_df_0.columns.values[i+1]]=np.array(merged_df.iloc[:,i+6]).astype(int)-np.array(merged_df.iloc[:,i+1]).astype(int)
final_df_1=final_df_1.iloc[np.argsort(np.array(final_df_1.company_name.apply(to_lower))),:]
final_df_1

,company_name,number_of_visits,number_of_distinct_user_visits,number_of_quick_link_visits,number_of_follows,number_of_saves
0,Affinity Management,54,39,3,4,6
1,Alcoa Inc.,5,1,2,0,0
2,American Express,81,46,14,15,7
3,ArcheMedX,2,2,0,0,1
4,Archer Roose,5,4,0,0,0
5,Bain & Company,77,51,5,22,12
6,BB&T,24,17,5,3,2
7,BizCorps,16,14,0,4,4
8,Castle Hill Gaming,2,1,0,0,0
9,Chevron,37,21,1,6,5


In [42]:
final_df_1.to_csv("Monthly_Company_Sites_Polularity (02-01-2017 to 02-28-2017).csv",index=False)